# TensorFlow SGD Lab

Welcome to the TensorFlow SGD Lab! By the end of this lab you will have

- Defined and optimized a TensorFlow computational graph to perform classification
- Used TensorBoard summary operations to visualize performance during training
- Used TensorBoard to visualize your computational graph

Let's get started!

---

# Load Data

The following code loads the [Boston housing price regression dataset](http://neupy.com/2015/07/04/boston_house_prices_dataset.html) via [keras](https://keras.io/datasets/) and discretizes the output space into four classes.

In [1]:
import numpy as np
from keras.datasets import boston_housing

nb_train = 400
np.random.seed(42)

[X_boston, y_boston], _ = boston_housing.load_data()
X_boston, y_boston = X_boston[:nb_train], y_boston[:nb_train]
sorted_idxs = np.argsort(y_boston)
X_boston, y_boston = X_boston[sorted_idxs], y_boston[sorted_idxs]
y_boston = np.hstack([np.full(shape=100, fill_value=i) for i in range(4)])
idxs = np.arange(nb_train)
np.random.shuffle(idxs)
X_boston, y_boston = X_boston[idxs], y_boston[idxs]

y_boston = np.expand_dims(y_boston, axis=-1)
X_boston.shape, y_boston.shape

Using TensorFlow backend.


((400, 13), (400, 1))

In [2]:
len(np.unique(y_boston))

4

In [3]:
def dense_to_one_hot(labels_dense, num_classes):
    """Convert class labels from scalars to one-hot vectors."""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

In [4]:
y_one_hot = dense_to_one_hot(y_boston,num_classes=len(np.unique(y_boston)))

In [5]:
y_one_hot[:5]

array([[ 0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.]])

In [6]:
X_boston.shape, y_one_hot.shape

((400, 13), (400, 4))

## Task

- Define and optimize a multiclass logistic regression classifier with TensorFlow

## Requirements

- Use TensorBoard to visualize your loss over time as well as your computational graph
- Perform optimization with stochastic gradient descent

## Deliverables

- Take a screenshot of your computational graph and loss in TensorBoard
- Remark on your optimized model on whether it fits the data well (e.g. underfits or overfits)
    - If your final model still does not fit the data well describe what you would try if you had more time and why

## Hints

- Attach a `tf.summary.scalar()` node to your loss node so TensorBoard can log its values over time

In [7]:
import tensorflow as tf

X = tf.placeholder(dtype=tf.float32, shape=[None, 13], name='X')
y = tf.placeholder(dtype=tf.float32, shape=[None, 4], name='y')
W = tf.Variable(initial_value=tf.zeros(shape=[13, 4]), name='W')
b = tf.Variable(initial_value=tf.zeros(shape=[4]), name='b')

z = tf.matmul(X, W, name='z')
y = tf.add(z, b, name='y')

y_pred = tf.placeholder(tf.float32, [None, 4], name='y_pred')
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_pred, logits=y))

In [8]:
loss_summary = tf.summary.scalar(name='loss_summary', tensor=cross_entropy)

In [9]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01, name='Optimizer')
sgd_step = optimizer.minimize(cross_entropy, name='SGDStep')

In [10]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

In [13]:
%%time

writer = tf.summary.FileWriter("tensorboard/LR with Loss Summary", sess.graph)
for i in range(20):
    ls, _ , W_, b_= sess.run([loss_summary, sgd_step, W, b], feed_dict={X: X_boston, y_pred: y_one_hot})
    print("W:",W_[0])
    print("b:",b_[0])
    writer.add_summary(ls, global_step=i)

writer.flush()

W: [-0.00846556 -0.0064471  -0.00237837  0.01729103]
b: 4.88944e-11
W: [-0.00748947 -0.00345255  0.0046849   0.00625711]
b: 0.0025
W: [-0.00651338 -0.00045799 -0.02601842  0.0329898 ]
b: 0.005
W: [-0.00553729 -0.03523004 -0.01895513  0.05972248]
b: 0.00749999
W: [-0.04232779 -0.03223549 -0.01189185  0.08645517]
b: 9.31323e-10
W: [-0.0413517  -0.02924094 -0.00482856  0.07542124]
b: 0.0025
W: [-0.04037561 -0.02624638 -0.03553189  0.10215392]
b: 0.005
W: [-0.03939952 -0.06101843 -0.0284686   0.12888661]
b: 0.0075
W: [-0.06547347 -0.05802388 -0.02140532  0.14490272]
b: 0.000650002
W: [-0.06449737 -0.05502933 -0.0143655   0.13389225]
b: 0.00315
W: [-0.06352128 -0.05203477 -0.04506882  0.16062494]
b: 0.00565
W: [-0.06254561 -0.07467472 -0.03800553  0.17522593]
b: 0.00812351
W: [-0.08648803 -0.07168017 -0.03094225  0.18911052]
b: 0.00142351
W: [-0.08551194 -0.06868648 -0.02400196  0.17820044]
b: 0.00392351
W: [-0.08453584 -0.06615535 -0.05424186  0.20493312]
b: 0.00642351
W: [-0.10615779 -0.0

### Loss Summary (Tensorboard)

<img src="loss_summary.png">

# Bonus Tasks

- Add hidden layers to your multiclass logistic regression classifier
- Use another optimizer such as adam or adagrad instead of stochastic gradient descent
- Monitor the accuracy